In [10]:
from __future__ import annotations

import json
import os

from dataclasses import dataclass
from typing import Any, Dict, List, Optional
from typing_extensions import Annotated, TypedDict

from neo4j import GraphDatabase
from langchain_core.tools import tool
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


os.environ["LANGSMITH_API_KEY"] = os.environ.get("LANGSMITH_API_KEY", "")
os.environ["LANGSMITH_TRACING"] = os.environ.get("LANGSMITH_TRACING", "true")


SYSTEM_PROMPT = """
Ти — агент штучного інтелекту з підтримкою виклику функцій для роботи з Neo4j-графом.
Тобі надаються підписи доступних інструментів у структурованому форматі.
Ти можеш викликати **лише один інструмент** — `search_graph_db`, щоб отримати дані з графа.
Не роби припущень і не вигадуй дані поза результатами виклику інструменту.

Опис графа:
Граф містить такі типи вузлів:
- Person (поля: first_name, full_name, last_name)
- Vehicle (поля: make, model, year)

Ось доступний інструмент:
[
  {
    "type": "function",
    "function": {
      "name": "search_graph_db",
      "description": "Виконує Cypher-запит до Neo4j-графа",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Cypher-запит для виконання"
          }
        },
        "required": ["query"]
      }
    }
  }
]

Використовуй наступну JSON schema (pydantic model) для кожного виклику інструменту:
{
  "title": "FunctionCall",
  "type": "object",
  "properties": {
    "name": {
      "title": "Name",
      "type": "string"
    },
    "arguments": {
      "title": "Arguments",
      "type": "object"
    }
  },
  "required": ["name", "arguments"]
}

Перед кожним викликом інструменту:
- Проаналізуй запит користувача
- Визнач, яку інформацію потрібно отримати з графа
- Сформуй коректний Cypher-запит

Правила використання `search_graph_db`:
- Обовʼязково використовуй інструмент, якщо користувач просить:
  факти з графа, списки, підрахунки, звʼязки, пошук сутностей
- Якщо є сумнів — виконай запит до графа, а не вгадуй

Правила написання Cypher-запитів:
- Використовуй лише READ-ONLY запити: MATCH ... RETURN ...
- Завжди додавай LIMIT (починай з 5–20)
- Повертай лише необхідні поля, а не цілі вузли без потреби
- Використовуй явні аліаси: RETURN p.name AS name
- Для агрегацій використовуй count(*), collect(...), DISTINCT, ORDER BY
- Якщо запит неоднозначний — постав ОДНЕ коротке уточнююче питання
  або виконай простий дослідницький запит

Після отримання результату:
- Формуй відповідь на основі JSON, який повернув інструмент
- Пояснюй відповідь лаконічно, вказуючи, з яких вузлів і полів вона отримана
- Якщо результат порожній — скажи, що нічого не знайдено,
  і запропонуй наступний запит або уточнення
- Ніколи не стверджуй, що виконав запит, якщо інструмент не був викликаний

Простий приклад:

Запит користувача:
"Знайди 5 людей та покажи їх імена"

Ти маєш викликати інструмент з таким Cypher-запитом:
MATCH (p:Person) RETURN p.name AS name LIMIT 5

Після цього прочитати JSON-відповідь та відповісти:
"Я знайшов стільки вузлів типу Person. Їхні імена: ..."

Нагадування: доступний лише один інструмент — `search_graph_db`.
"""


@dataclass(frozen=True)
class AgentConfig:
    model_name: str
    base_url: str
    api_key: str
    temperature: float = 0.0

    neo4j_uri: str = "bolt://localhost:7687"
    neo4j_user: str = "neo4j"
    neo4j_password: str = "password"
    neo4j_database: Optional[str] = None

    allow_write_queries: bool = False


def make_search_graph_db_tool(cfg: AgentConfig):
    @tool("search_graph_db")
    def search_graph_db(query: str) -> str:
        """
        Query the Neo4j graph database using Cypher queries.
        Returns JSON string with query results (list of records as dicts).
        """
        q = (query or "").strip()
        if not q:
            return json.dumps({"error": "Empty query"}, ensure_ascii=False)

        if not cfg.allow_write_queries:
            blocked = ("CREATE", "MERGE", "DELETE", "SET", "DROP", "CALL", "LOAD CSV", "APOC")
            upper = q.upper()
            if any(tok in upper for tok in blocked):
                return json.dumps(
                    {
                        "error": "Write/procedure queries are disabled for this agent.",
                        "hint": "Use MATCH/RETURN (read-only) queries.",
                    },
                    ensure_ascii=False,
                )

        try:
            auth = (cfg.neo4j_user, cfg.neo4j_password)
            with GraphDatabase.driver(cfg.neo4j_uri, auth=auth) as driver:
                sess_kwargs: Dict[str, Any] = {}
                if cfg.neo4j_database:
                    sess_kwargs["database"] = cfg.neo4j_database

                with driver.session(**sess_kwargs) as session:
                    result = session.run(q)
                    records: List[Dict[str, Any]] = [r.data() for r in result]
                    return json.dumps(records, ensure_ascii=False)
        except Exception as exc:
            return json.dumps({"error": f"Neo4j query failed: {exc}"}, ensure_ascii=False)

    return search_graph_db


class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]


class Neo4jLangGraphAgent:
    def __init__(self, cfg: AgentConfig, system_prompt: Optional[str] = None):
        self.cfg = cfg
        self.system_prompt = system_prompt or SYSTEM_PROMPT

        self.tool = make_search_graph_db_tool(cfg)

        self.llm = ChatOpenAI(
            model=cfg.model_name,
            base_url=cfg.base_url,
            api_key=cfg.api_key,
            temperature=cfg.temperature,
        ).bind_tools([self.tool], tool_choice="any")

        self.graph = self._build_graph()

    def _build_graph(self):
        tool_node = ToolNode([self.tool])

        def assistant_node(state: AgentState) -> Dict[str, Any]:
            resp = self.llm.invoke(state["messages"])
            return {"messages": [resp]}

        g = StateGraph(AgentState)
        g.add_node("assistant", assistant_node)
        g.add_node("tools", tool_node)

        g.add_edge(START, "assistant")
        g.add_conditional_edges("assistant", tools_condition, {"tools": "tools", END: END})
        g.add_edge("tools", "assistant")

        return g.compile()

    def invoke(self, user_text: str) -> str:
        init_messages: List[AnyMessage] = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=user_text),
        ]
        out = self.graph.invoke({"messages": init_messages})
        return out["messages"][-1].content


In [11]:
cfg = AgentConfig(
    model_name="lapa-function-calling",              
    base_url=os.getenv("BASE_URL", ""),   
    api_key=os.getenv("API_KEY", ""),
    neo4j_uri=os.getenv("NEO4J_URI", ""),
    neo4j_user="neo4j",
    neo4j_password=os.getenv("NEO4J_PASSWORD", ""),
    neo4j_database="neo4j",                     
)

agent = Neo4jLangGraphAgent(cfg)
print(agent.graph.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
        +-----------+         
        | assistant |         
        +-----------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


In [12]:
answer = agent.invoke(
    "Виведи всіх людей та їх прізвища"
)

print(answer)

BadRequestError: Error code: 400 - {'error': {'message': 'litellm.BadRequestError: OpenAIException - Conversation roles must alternate user/assistant/user/assistant/... Conversation roles must alternate user/assistant/user/assistant/.... Received Model Group=lapa-function-calling\nAvailable Model Group Fallbacks=None', 'type': None, 'param': None, 'code': '400'}}

# Testing tool calling

In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Raw OpenAI client
client = OpenAI(
    api_key="YOU_API_KEY",
    base_url="https://api.lapathoniia.top"
)

# LangChain tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

def test_raw_openai():
    print("=== Testing Raw OpenAI Client ===")
    
    tools = [{
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer"},
                    "b": {"type": "integer"}
                },
                "required": ["a", "b"]
            }
        }
    }]
    
    response = client.chat.completions.create(
        model="lapa-function-calling",
        messages=[{"role": "user", "content": "Add 15 and 27 using the function"}],
        tools=tools,
        tool_choice="required"  # Force function calling
    )
    
    print(f"Response: {response.choices[0].message}")
    if response.choices[0].message.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.choices[0].message.tool_calls:
            print(f"Function: {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
            
            # Execute the function
            import json
            args = json.loads(tool_call.function.arguments)
            result = args['a'] + args['b']
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")

def test_langchain():
    print("\n=== Testing LangChain ===")
    
    llm = ChatOpenAI(
        model="lapa-function-calling",
        api_key="sk-X_GR0JPbgYsIVRspba2ArA",
        base_url="https://api.lapathoniia.top"
    )
    
    llm_with_tools = llm.bind_tools([add_numbers], tool_choice="any")
    
    response = llm_with_tools.invoke([HumanMessage(content="Use the add_numbers function to add 15 and 27")])
    
    print(f"Tool calls: {response.tool_calls}")
    if response.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.tool_calls:
            print(f"Function: {tool_call['name']}")
            print(f"Arguments: {tool_call['args']}")
            
            # Execute the function
            result = add_numbers.invoke(tool_call['args'])
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")
        print(f"Text response: {response.content}")

In [ ]:
test_langchain()